 **In this notebook,I try to analysis house price dataset and predict the house prices by regression methods.**
 **I start with introdusing the dataset,then I do data analysis and house price prediction step by step.**

# **Import the dataset:**

**This dataset have 1460 rows and 81 columns.The SalePrice is the target variable that I am trying to predict.At first I should import all necessary libraries and then read and import the dataset:**

In [ ]:
import numpy as np #Import all necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
train=pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")

In [ ]:
train.head()

In [ ]:
#Cleaning the Id columns because it doesnt have important information.
train=train.drop("Id",axis=1)

In [ ]:
train.info()

### **Target Value**

In [ ]:
fig=plt.figure(figsize=(8,4),dpi=100)# Draw histogram for target variable
sns.distplot(train["SalePrice"],hist_kws=dict(edgecolor="w",linewidth=1),bins=25,color="r")
plt.title("Sales data distribution")

**As we see the most amount of prices are between 100000 to 300000.This SalePrice distribution is not very normal so I use log for that to make more normal distribution.**

In [ ]:
train['saleprice'] = np.log1p(train['SalePrice'])#Use log function in numpy

fig=plt.figure(figsize=(8,4),dpi=100)
sns.distplot(train["saleprice"],hist_kws=dict(edgecolor="w",linewidth=1),bins=25,color="r")
plt.title("Sales data distribution")

### **Checking Missing Data**
**Before I continue,I want to introduce the missing value briefly.The real-world data often has a lot of missing values. The cause of missing values can be data corruption or failure to record data.
The handling of missing data is very important during the preprocessing of the dataset as many machine learning algorithms do not support missing values.**
**Now I want to identify the rows with the most number of missing values and drop or transform them.**

In [ ]:
#Let's check if the data set has any missing values.
100*(train.isnull().sum()/len(train))
def missing_values_percent(train):#we can use this function in all dataframes.
    nan_percent=100*(train.isnull().sum()/len(train))
    nan_percent=nan_percent[nan_percent>0].sort_values()
    return(nan_percent)



In [ ]:
nan_percent=missing_values_percent(train)
nan_percent

In [ ]:
#Drawing barplot for these missing values.
plt.figure(figsize=(12,6))
sns.barplot(x=nan_percent.index,y=nan_percent)
plt.xticks(rotation=45)

In [ ]:
#determine the missing values that their percentages are between 0 and 5.
plt.figure(figsize=(12,6))
sns.barplot(x=nan_percent.index,y=nan_percent)
plt.xticks(rotation=45)

plt.ylim(0,5)

## **In this step we want to make a decision about our missing data**

In [ ]:
train[train["Electrical"].isnull()]

In [ ]:
#can delet this row
train=train.drop(labels=1379,axis=0)

In [ ]:
nan_percent=missing_values_percent(train)#see Electrical has been droped from missing data.
plt.figure(figsize=(12,6))
sns.barplot(x=nan_percent.index,y=nan_percent)
plt.xticks(rotation=45)

plt.ylim(0,5)

In [ ]:
train["MasVnrType"]=train["MasVnrType"].fillna("None")
train["MasVnrArea"]=train["MasVnrArea"].fillna(0)

In [ ]:
nan_percent=missing_values_percent(train)#see MasVnrType and MasVnrArea have been droped.
plt.figure(figsize=(12,6))
sns.barplot(x=nan_percent.index,y=nan_percent)
plt.xticks(rotation=45)

plt.ylim(0,5)

In [ ]:
bsmt_str_cols=["BsmtQual","BsmtCond","BsmtFinType1","BsmtExposure","BsmtFinType2"]
train[bsmt_str_cols]=train[bsmt_str_cols].fillna("None")

In [ ]:
gar_str_cols=["GarageType","GarageFinish"]#transform the object to None
train[gar_str_cols]=train[gar_str_cols].fillna("None")

gar_num_cols=["GarageYrBlt","GarageQual","GarageCond"]#transform the int or float to 0
train[gar_num_cols]=train[gar_num_cols].fillna(0)

In [ ]:
nan_percent=missing_values_percent(train)#see MasVnrType and MasVnrArea have been droped.
plt.figure(figsize=(12,6))
sns.barplot(x=nan_percent.index,y=nan_percent)
plt.xticks(rotation=45)

In [ ]:
train["FireplaceQu"]=train["FireplaceQu"].fillna("None")

**From the below boxplot we understand that any neighborhood have specefic LotFrontage distribution.So we can replace the average of Lotfrontage base on each Neighborhood with the missing data.**

In [ ]:
plt.figure(figsize=(8,12))
sns.boxplot(data=train,x="LotFrontage",y="Neighborhood")


In [ ]:
train.groupby("Neighborhood")["LotFrontage"].mean()#average of Lotfrontage base on each Neighborhood

In [ ]:
#replace the average of Lotfrontage base on each Neighborhood with the missing data
train["LotFrontage"]=train.groupby("Neighborhood")["LotFrontage"].transform(lambda val:val.fillna(val.mean()))

In [ ]:
nan_percent=missing_values_percent(train)#see MasVnrType and MasVnrArea have been droped.
plt.figure(figsize=(12,6))
sns.barplot(x=nan_percent.index,y=nan_percent)
plt.xticks(rotation=45)

**As we see,we have only 4 features that have missing data.As the percentage of missing datas in these features are above 80%,the best way is to drop them from the dataframe.**

In [ ]:
train=train.drop(["Fence","Alley","MiscFeature","PoolQC"],axis=1)

In [ ]:
train.isnull().sum()#we have no missing data

## Now,we have no missing data.

In [ ]:
train.corr()["saleprice"]#determine correlation

# **Exploratory data analysis**

**This is not a very big data and we do not have too many features. Thus, we have chance to plot most of them and reach some useful analytical results. Drawing charts and examining the data before applying a model is a very good practice because we may detect some possible outliers or decide to do normalization.**

In [ ]:
#we want to determine the relationship between saleprice and yearsold.
train.groupby("YrSold")["saleprice"].mean().plot(color="r")

### We see that the price is decreasing between 2006 to 2010,The highest saleprice belongs to 2007 then we have the sharp decreasing in 2008,I think it is weired because the price increases mainly over the years.

In [ ]:
corr = train.corr()#Top 50% Corralation train attributes with sale-price
top_feature = corr.index[abs(corr['saleprice']>0.5)]
plt.subplots(figsize=(12, 8))
top_corr = train[top_feature].corr()
sns.heatmap(top_corr, annot=True)

### As we see ten features have the most value of correlation,**overallqual** with alomost 80% has the most value of correlation and the second one is Grlivearea with almost 70%.

## NOW I want to plot the 2 top features:

In [ ]:
sns.scatterplot(data=train,x="OverallQual",y="saleprice")


**We see that the customer should pay more for high overallqual and they dont prefer to buy a house with the quality under 3.**

In [ ]:
sns.scatterplot(data=train,x="GrLivArea",y="saleprice")
plt.axhline(y=11.5 , color="r")
plt.axvline(x=4000,color="r")

# we see that we have 4 *outliers* which we decide about them.

In [ ]:
train[(train["GrLivArea"]>4000) & (train["saleprice"]>11.5)][["GrLivArea","saleprice"]]#determine the index of outliers

In [ ]:
index_drop=train[(train["GrLivArea"]>4000) & (train["saleprice"]>11.5)].index#drop the outliers
train=train.drop(index_drop,axis=0)

### Further, I drew the 3D plot to determine the relation between the saleprice, overallqual and year built. The below chart shows that the newer houses have the more overalqual but we can not say this about fullbath.

In [ ]:

from mpl_toolkits.mplot3d import Axes3D


fig = plt.figure(figsize=(8,8))

ax = Axes3D(fig) 

x = train["YearBuilt"]
y = train["FullBath"]
z = train["OverallQual"]


ax.scatter(x, y, z, c=x, marker='o')
ax.set_xlabel('YearBuilt')
ax.set_ylabel('FullBath')
ax.set_zlabel('OverallQual')


### **Categorical data**

**In this step,I want to encoding str to int.**

In [ ]:
#At first,I tranform MSSubClass to str:
train["MSSubClass"].apply(str)

In [ ]:
train.select_dtypes(include="object")#seperate the data that are object

In [ ]:
#Divide dataframe to 2 parts(num and str)
train_num=train.select_dtypes(exclude="object")
train_obj=train.select_dtypes(include="object")

In [ ]:
train_obj=pd.get_dummies(train_obj,drop_first=True)#use one-hot encoding to transform str to int and float
train_obj.shape

In [ ]:
Final_train=pd.concat([train_num,train_obj],axis=1)

# **LinearRegression**

**Linear regression is probably one of the most important and widely used regression techniques. It’s among the simplest regression methods. One of its main advantages is the ease of interpreting results.
Linear regression performs the task to predict a dependent variable value (y) based on a given independent variable (x). So, this regression technique finds out a linear relationship between x (input) and y(output). Hence, the name is Linear Regression.**
**In this step we want to predict our target valuable (saleprice) based on our features.**

 **After import the data,data overview and exploratory data analysis which are shown,we should determine the feature and lable.**

In [ ]:
#Determine the feature and lable
X=Final_train.drop("saleprice",axis=1)
y=Final_train["saleprice"]

In [ ]:
#Split the dataset to train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)

In [ ]:
#train the model
from sklearn.linear_model import LinearRegression
model=LinearRegression()
model.fit(X_train, y_train)

In [ ]:
#coeficient matrix
pd.DataFrame(model.coef_,X.columns,columns=["coeficient"])

In [ ]:
#predicting test data
y_pred=model.predict(X_test)

In [ ]:
#evaluating the model
from sklearn import metrics
MAD=metrics.mean_absolute_error(y_test,y_pred)
MSE=metrics.mean_squared_error(y_test,y_pred)
RMSE=np.sqrt(MSE)

In [ ]:
pd.DataFrame(data=[MAD,MSE,RMSE],index=["MAD","MSE","RMSE"],columns=["LinearRegression"])

## **Residuals**

**For linear regression it is a good idea to evaluate residuals(y- ̂y)**


In [ ]:
test_residuals=y_test-y_pred#the residuals should be random and close to normal distribution.
sns.scatterplot(x=y_test,y=y_pred,color="r")
plt.xlabel('Y-Test')
plt.ylabel('Y-Pred')

In [ ]:
sns.scatterplot(x=y_test,y=test_residuals,color="r")#test residuals should not show a clear pattern.
plt.axhline(y=0,color="b",ls="--")
plt.xlabel('Y-Test')
plt.ylabel('residuals')

 ### **As we see ,the residuals show a clear pattern.so we can say,linear regression is not suitable for this model.**

# **Ridge regression**

**Ridge regression is a model tuning method that is used to analyse any data that suffers from multicollinearity. This method performs L2 regularization. When the issue of multicollinearity occurs, least-squares are unbiased, and variances are large, this results in predicted values to be far away from the actual values.**

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
ridge_model=Ridge(alpha=10)

In [ ]:
ridge_model.fit(X_train,y_train)

Ridge regression

In [ ]:
y_pred=ridge_model.predict(X_test)

In [ ]:
from sklearn import metrics
MAD3=metrics.mean_absolute_error(y_test,y_pred)
MSE3=metrics.mean_squared_error(y_test,y_pred)
RMSE3=np.sqrt(MSE3)

In [ ]:
data={'LinearRegression':[MAD,MSE,RMSE],"Ridge regression":[MAD3,MSE3,RMSE3]}
pd.DataFrame(data,index=["MAD","MSE","RMSE"])

## **Ridgecv(cross validation)**

**Cross-validation** is a statistical method used to estimate the skill of machine learning models.

It is commonly used in applied machine learning to compare and select a model for a given predictive modeling problem because it is easy to understand, easy to implement, and results in skill estimates that generally have a lower bias than other methods.

In [ ]:
from sklearn.linear_model import RidgeCV

In [ ]:
from sklearn.linear_model import RidgeCV

In [ ]:
ridge_cv_model=RidgeCV(alphas=(0.1, 1.0, 10.0),scoring="neg_mean_absolute_error")

In [ ]:
ridge_cv_model.fit(X_train,y_train)

In [ ]:
ridge_cv_model.alpha_

In [ ]:
y_pred_ridge=ridge_cv_model.predict(X_test)

In [ ]:
from sklearn import metrics
MAD4=metrics.mean_absolute_error(y_test,y_pred_ridge)
MSE4=metrics.mean_squared_error(y_test,y_pred_ridge)
RMSE4=np.sqrt(MSE4)

In [ ]:
data={'LinearRegression':[MAD,MSE,RMSE],"Ridge regression":[MAD3,MSE3,RMSE3],"Ridgecv":[MAD4,MSE4,RMSE4]}
pd.DataFrame(data,index=["MAD","MSE","RMSE"])

# **Lasso regression**

**Lasso regression is a regularization technique. It is used over regression methods for a more accurate prediction. This model uses shrinkage. Shrinkage is where data values are shrunk towards a central point as the mean. The lasso procedure encourages simple, sparse models (i.e. models with fewer parameters)**

In [ ]:
from sklearn.linear_model import LassoCV

In [ ]:
lasso_cv_model=LassoCV(eps=0.1,n_alphas=100,cv=5)

In [ ]:
lasso_cv_model.fit(X_train,y_train)

In [ ]:
lasso_cv_model.alpha_

In [ ]:
y_pred_lasso=lasso_cv_model.predict(X_test)

In [ ]:
from sklearn import metrics
MAD5=metrics.mean_absolute_error(y_test,y_pred_lasso)
MSE5=metrics.mean_squared_error(y_test,y_pred_lasso)
RMSE5=np.sqrt(MSE5)

In [ ]:
data={'LinearRegression':[MAD,MSE,RMSE],"Ridge regression":[MAD3,MSE3,RMSE3],"Ridgecv":[MAD4,MSE4,RMSE4],"Lassocv":[MAD5,MSE5,RMSE5]}
pd.DataFrame(data,index=["MAD","MSE","RMSE"])

# **Elastic Net**

**Elastic net is a popular type of regularized linear regression that combines two popular penalties, specifically the L1 and L2 penalty functions. ... Elastic Net is an extension of linear regression that adds regularization penalties to the loss function during training.**

In [ ]:
from sklearn .linear_model import ElasticNetCV
elastic_model=ElasticNetCV(l1_ratio=[0.1,0.5,0.7,0.9,0.95,0.99,1],cv=5,max_iter=100000)

In [ ]:
elastic_model.fit(X_train,y_train)
elastic_model.l1_ratio_

In [ ]:
y_pred_elastic=elastic_model.predict(X_test)

In [ ]:
from sklearn import metrics
MAD6=metrics.mean_absolute_error(y_test,y_pred_elastic)
MSE6=metrics.mean_squared_error(y_test,y_pred_elastic)
RMSE6=np.sqrt(MSE6)

In [ ]:
data={'LinearRegression':[MAD,MSE,RMSE],"Ridge regression":[MAD3,MSE3,RMSE3],"Ridgecv":[MAD4,MSE4,RMSE4],"Lassocv":[MAD5,MSE5,RMSE5],"elasticnet":[MAD6,MSE6,RMSE6]}
pd.DataFrame(data,index=["MAD","MSE","RMSE"])

# **From my opinin,as the residuals show the clear pattern in linear regression,according to the RMSE error the ridgecv is the best method in this model.**